In [1]:
import pandas as pd

submission = pd.read_csv('../input/feedback-prize-2021/sample_submission.csv')

In [27]:
submission

,id,class,predictionstring
0,18409261F5C2,NaN,NaN
1,D46BCB48440A,NaN,NaN
2,0FB0700DAF44,NaN,NaN
3,D72CB1C11673,NaN,NaN
4,DF920E0A7337,NaN,NaN


In [9]:
from tqdm import tqdm
import os
test_names, test_texts = [], []
test_texts_list = []
for f in tqdm(list(os.listdir('../input/feedback-prize-2021/test'))):
    test_names.append(f.replace('.txt', ''))
    test_texts.append(open('../input/feedback-prize-2021/test/' + f, 'r', encoding='utf-8').read())
for t in test_texts:
    test_texts_list.append(t.split())
test_text_df = pd.DataFrame({'id': test_names, 'text': test_texts, 'text_list': test_texts_list})

100%|██████████| 5/5 [00:00<00:00, 3267.61it/s]


In [10]:
test_text_df

,id,text,text_list
0,D46BCB48440A,"When people ask for advice,they sometimes talk...","[When, people, ask, for, advice,they, sometime..."
1,DF920E0A7337,Have you ever asked more than one person for h...,"[Have, you, ever, asked, more, than, one, pers..."
2,0FB0700DAF44,"During a group project, have you ever asked a ...","[During, a, group, project,, have, you, ever, ..."
3,D72CB1C11673,Making choices in life can be very difficult. ...,"[Making, choices, in, life, can, be, very, dif..."
4,18409261F5C2,80% of Americans believe seeking multiple opin...,"[80%, of, Americans, believe, seeking, multipl..."


In [11]:
from datasets import Dataset

test_datasets = Dataset.from_pandas(test_text_df)

In [12]:
test_datasets

Dataset({
    features: ['id', 'text', 'text_list'],
    num_rows: 5
})

In [15]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('allenai/longformer-base-4096', add_prefix_space=True)

def preparing_test_dataset(examples):
    encoding = tokenizer(examples['text_list'], truncation=True, padding=False, max_length = 2048, is_split_into_words=True)
    total= len(encoding['input_ids'])
    encoding['word_ids']=[]
    encoding['id']=[]
    for i in range(total):
        word_idx = encoding.word_ids(batch_index=i)
        encoding['word_ids'].append(word_idx)
        encoding['id'].append(examples['id'][i])
    return encoding

tokenized_test_datasets = test_datasets.map(preparing_test_dataset, batched=True, batch_size=5, remove_columns=test_datasets.column_names)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [19]:
from transformers import AutoModelForTokenClassification, AutoConfig, Trainer, DataCollatorForTokenClassification
import numpy as np
data_collator = DataCollatorForTokenClassification(tokenizer)
all_predictions = []
for fold in range(5):
    model_path = './best_model/longformer-baseline_fold'+str(fold)+'/'
    config = AutoConfig.from_pretrained(model_path)
    model = AutoModelForTokenClassification.from_pretrained(model_path, config = config)
    
    trainer = Trainer(
        model = model,
        train_dataset=None,
        eval_dataset=tokenized_test_datasets,
        tokenizer=tokenizer,
        data_collator=data_collator,
    )
    predictions, _, _ = trainer.predict(test_dataset = tokenized_test_datasets)
    print("shape of prediciton", predictions.shape)
    all_predictions.append(predictions.astype(np.float32))

loading configuration file ./best_model/longformer-baseline_fold0/config.json
Model config LongformerConfig {
  "_name_or_path": "./best_model/longformer-baseline_fold0/",
  "architectures": [
    "LongformerForTokenClassification"
  ],
  "attention_mode": "longformer",
  "attention_probs_dropout_prob": 0.1,
  "attention_window": [
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14"
  },
  "ignore_attention_mask": false,
  "init

loading configuration file ./best_model/longformer-baseline_fold1/config.json
Model config LongformerConfig {
  "_name_or_path": "./best_model/longformer-baseline_fold1/",
  "architectures": [
    "LongformerForTokenClassification"
  ],
  "attention_mode": "longformer",
  "attention_probs_dropout_prob": 0.1,
  "attention_window": [
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14"
  },
  "ignore_attention_mask": false,
  "init

shape of prediciton (5, 1292, 15)


All model checkpoint weights were used when initializing LongformerForTokenClassification.

All the weights of LongformerForTokenClassification were initialized from the model checkpoint at ./best_model/longformer-baseline_fold1/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use LongformerForTokenClassification for predictions without further training.
No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the test set  don't have a corresponding argument in `LongformerForTokenClassification.forward` and have been ignored: id, word_ids.
***** Running Prediction *****
  Num examples = 5
  Batch size = 8
Input ids 

loading configuration file ./best_model/longformer-baseline_fold2/config.json
Model config LongformerConfig {
  "_name_or_path": "./best_model/longformer-baseline_fold2/",
  "architectures": [
    "LongformerForTokenClassification"
  ],
  "attention_mode": "longformer",
  "attention_probs_dropout_prob": 0.1,
  "attention_window": [
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14"
  },
  "ignore_attention_mask": false,
  "init

...

All model checkpoint weights were used when initializing LongformerForTokenClassification.

All the weights of LongformerForTokenClassification were initialized from the model checkpoint at ./best_model/longformer-baseline_fold3/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use LongformerForTokenClassification for predictions without further training.
No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the test set  don't have a corresponding argument in `LongformerForTokenClassification.forward` and have been ignored: id, word_ids.
***** Running Prediction *****
  Num examples = 5
  Batch size = 8
Input ids 

loading configuration file ./best_model/longformer-baseline_fold4/config.json
Model config LongformerConfig {
  "_name_or_path": "./best_model/longformer-baseline_fold4/",
  "architectures": [
    "LongformerForTokenClassification"
  ],
  "attention_mode": "longformer",
  "attention_probs_dropout_prob": 0.1,
  "attention_window": [
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14"
  },
  "ignore_attention_mask": false,
  "init

shape of prediciton (5, 1292, 15)


All model checkpoint weights were used when initializing LongformerForTokenClassification.

All the weights of LongformerForTokenClassification were initialized from the model checkpoint at ./best_model/longformer-baseline_fold4/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use LongformerForTokenClassification for predictions without further training.
No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the test set  don't have a corresponding argument in `LongformerForTokenClassification.forward` and have been ignored: id, word_ids.
***** Running Prediction *****
  Num examples = 5
  Batch size = 8
Input ids 

shape of prediciton (5, 1292, 15)


In [21]:
x=0
for preds in all_predictions:
    preds = preds/5
    x+=preds

In [25]:
from utils_fb import postprocess_fb_predictions

oof = postprocess_fb_predictions(eval_examples=None, eval_datasets=tokenized_test_datasets, predictions=x)


(5, 1292, 15)


5it [00:00, 291.36it/s]


In [26]:
oof

,id,class,predictionstring
0,D46BCB48440A,Evidence,56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 7...
1,D46BCB48440A,Evidence,150 151 152 153 154 155 156 157 158 159 160 16...
2,D46BCB48440A,Evidence,223 224 225 226 227 228 229 230 231 232 233 23...
3,DF920E0A7337,Evidence,118 119 120 121 122 123 124 125 126 127 128 12...
4,DF920E0A7337,Evidence,310 311 312 313 314 315 316 317 318 319 320 32...
5,DF920E0A7337,Evidence,466 467 468 469 470 471 472 473 474 475 476 47...
6,0FB0700DAF44,Evidence,120 121 122 123 124 125 126 127 128 129 130 13...
7,0FB0700DAF44,Evidence,341 342 343 344 345 346 347 348 349 350 351 35...
8,D72CB1C11673,Evidence,94 95 96 97 98 99 100 101 102 103 104 105 106 ...
9,D72CB1C11673,Evidence,183 184 185 186 187 188 189 190 191 192 193 19...


In [28]:
oof.to_csv("submission.csv", index=False)